# Intro
- This is the first jupyter notebook for the EIA interhemispheric assymmetry (IHA) seen during different solar cycles
- TEC comparison during the March equinox is carried out

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import cartopy.crs as ccrs
from cartopy.feature.nightshade import Nightshade
from mpl_toolkits.axes_grid1 import make_axes_locatable

from tqdm import tqdm
import apexpy
from apexpy import Apex

import os
import importlib as il
import datetime as dt
from scipy.interpolate import CubicSpline

import sys
sys.path.append('../')
from utility_programs import filters
from scipy.signal import find_peaks, savgol_filter
from multiprocessing import Pool, cpu_count

sys.path.append('/home/pxv220016/prasoon/data/sat_interp_repo/repo2/prasoon_utility_programs')

In [13]:
path = '/home/pxv220016/scratch/Qingyu_Cesar_EIA/'
files = os.listdir(path)

tec_g = []
columns = ['GDLAT', 'GLON', 'TEC', 'DT']
grnd_tec = pd.DataFrame(columns=columns)

for f in tqdm(files):
    df = pd.read_csv(path+f, sep=r'\s+')
    d = [dt.datetime(y,m,d,h,mi,s) for y,m,d,h,mi,s in zip(df.YEAR, df.MONTH, df.DAY, df.HOUR, df.MIN, df.SEC)]
    df['DT'] = d
    df = df.drop(['RECNO','KINDAT','KINST','UT1_UNIX','UT2_UNIX','YEAR','MONTH','DAY','HOUR','MIN','SEC','DTEC'], axis=1)
    if 'GDALT' in df.columns:
        df = df.drop(['GDALT'],axis=1)
    tec_g.append(df)
    grnd_tec = pd.concat([grnd_tec, df], axis = 0)

grnd_tec.reset_index(drop=True, inplace=True)
print(len(tec_g))
print(grnd_tec)


100%|███████████████████████████████████████████| 44/44 [17:51<00:00, 24.35s/it]

44
           GDLAT   GLON   TEC                  DT
0          -90.0 -124.0  12.0 2014-03-20 00:02:30
1          -90.0 -123.0  11.9 2014-03-20 00:02:30
2          -90.0 -122.0  11.8 2014-03-20 00:02:30
3          -90.0  -33.0  13.2 2014-03-20 00:02:30
4          -90.0  -31.0  13.0 2014-03-20 00:02:30
...          ...    ...   ...                 ...
188069583   88.0   90.0  10.9 2014-03-01 23:57:30
188069584   88.0   92.0  10.8 2014-03-01 23:57:30
188069585   88.0  148.0  11.8 2014-03-01 23:57:30
188069586   88.0  149.0  11.7 2014-03-01 23:57:30
188069587   89.0  149.0  11.7 2014-03-01 23:57:30

[188069588 rows x 4 columns]


In [27]:
def magnetic_coords(sat_data):
    mlat = []
    mlon = []
    mlt = []
    
    # Introducing the MLAT, MLON, MLT columns in satlocddf
    for glat, glon, gtime in tqdm(zip(sat_data.GDLAT, sat_data.GLON, sat_data.DT), total=len(sat_data.GLON)):
    
        decimal_year = gtime.year + ((gtime - dt.datetime(gtime.year, 1, 1)).days)/365

        apex = Apex(decimal_year)
        mlatitude, mlongitude = apex.convert(glat, glon, 'geo', 'apex')
        mtime = apex.mlon2mlt(mlongitude, gtime)
        #mlatitude, mlongitude = magfield
        mlat.append(mlatitude)
        mlon.append(mlongitude)
        mlt.append(mtime)

    # Adding mlat and mlon data variables in dataset
    sat_data['MLAT'] = mlat
    sat_data['MLON'] = mlon
    sat_data['MLT'] = mlt
    
    return sat_data


In [29]:
grnd_tec = magnetic_coords(grnd_tec)
print(grnd_tec)

  1%|▏                         | 1682404/188069588 [11:24<21:04:03, 2457.53it/s]


KeyboardInterrupt: 